In [33]:
link = '../../vid/drone-flight.mp4'
#!/usr/bin/env python

'''
Camshift tracker
================

This is a demo that shows mean-shift based tracking
You select a color objects such as your face and it tracks it.
This reads from video camera (0 by default, or the camera number the user enters)

http://www.robinhewitt.com/research/track/camshift.html

Usage:
------
    camshift.py [<video source>]

    To initialize tracking, select the object with mouse

Keys:
-----
    ESC   - exit
    b     - toggle back-projected probability visualization
'''

# Python 2/3 compatibility
from __future__ import print_function
import sys
PY3 = sys.version_info[0] == 3

if PY3:
    xrange = range

import numpy as np
import cv2 as cv

# local module
import video
from video import presets


class App(object):
    def __init__(self, video_src):
        self.cam = video.create_capture(video_src, presets['cube'])
        _ret, self.frame = self.cam.read()
        cv.namedWindow('camshift')
        cv.setMouseCallback('camshift', self.onmouse)

        self.selection = None
        self.drag_start = None
        self.show_backproj = False
        self.track_window = None

    def onmouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.track_window = None
        if self.drag_start:
            xmin = min(x, self.drag_start[0])
            ymin = min(y, self.drag_start[1])
            xmax = max(x, self.drag_start[0])
            ymax = max(y, self.drag_start[1])
            self.selection = (xmin, ymin, xmax, ymax)
        if event == cv.EVENT_LBUTTONUP:
            self.drag_start = None
            self.track_window = (xmin, ymin, xmax - xmin, ymax - ymin)

    def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)
        for i in xrange(bin_count):
            h = int(self.hist[i])
            cv.rectangle(img, (i*bin_w+2, 255), ((i+1)*bin_w-2, 255-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv.cvtColor(img, cv.COLOR_HSV2BGR)
        cv.imshow('hist', img)

    def run(self):
        while True:
            _ret, self.frame = self.cam.read()
            vis = self.frame.copy()
            hsv = cv.cvtColor(self.frame, cv.COLOR_BGR2HSV)
            mask = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
            if self.selection:
                x0, y0, x1, y1 = self.selection
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                hist = cv.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
                # hist = np.array([0, 0, 0., 0., 0., 0., 0., 255, 183.42105, 0., 0., 0., 0., 0., 0., 0.])
                cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                print(self.hist)
                self.show_hist()

                vis_roi = vis[y0:y1, x0:x1]
                cv.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            if self.track_window and self.track_window[2] > 0 and self.track_window[3] > 0:
                self.selection = None
                prob = cv.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                prob &= mask
                term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
                track_box, self.track_window = cv.CamShift(prob, self.track_window, term_crit)

                if self.show_backproj:
                    vis[:] = prob[...,np.newaxis]
                try:
                    cv.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    print(track_box)

            cv.imshow('camshift', vis)

            ch = cv.waitKey(5)
            if ch == 27:
                break
            if ch == ord('b'):
                self.show_backproj = not self.show_backproj
        cv.destroyAllWindows()


if __name__ == '__main__':
    import sys
    try:
        sys.argv[1] = link
        video_src = sys.argv[1]
    except:
        video_src = 0
    print(__doc__)
    App(video_src).run()



Camshift tracker

This is a demo that shows mean-shift based tracking
You select a color objects such as your face and it tracks it.
This reads from video camera (0 by default, or the camera number the user enters)

http://www.robinhewitt.com/research/track/camshift.html

Usage:
------
    camshift.py [<video source>]

    To initialize tracking, select the object with mouse

Keys:
-----
    ESC   - exit
    b     - toggle back-projected probability visualization

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[  0.         0.         0.         0.         0.         0.
   0.         0.       254.99998  142.67856   13.154761  11.130952
  23.273808   4.047619   0.         0.      ]
[  0.         0.         0.         0.         0.         0.
   0.        11.992574 254.99998  153.37871   31.559404  25.247524
  24.616335   0.         0.         0.      ]
[  0.          0.          0.          0.          0.          0.
   0.          4

In [13]:
hist = np.array([0, 0, 0., 0., 0., 0., 0., 255, 183.42105, 0., 0., 0., 0., 0., 0., 0.])
cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
hist.reshape(-1)

array([  0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     , 255.     , 183.42105,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ])

In [16]:
hist.reshape(-1)

array([  0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
         0.     , 255.     , 183.42105,   0.     ,   0.     ,   0.     ,
         0.     ,   0.     ,   0.     ,   0.     ])

In [20]:

ret, frame = cap.read()
y, x, rgb = frame.shape
# xmin, ymin = 0, 0
# xmax, ymax = x, y
xmin, ymin = 800, 500
xmax, ymax = 850, 600
selection = (xmin, ymin, xmax, ymax)
track_window = (xmin, ymin, xmax - xmin, ymax - ymin)
show_backproj = False

x0, y0, x1, y1 = selection
# hsv_roi = hsv[y0:y1, x0:x1]
# mask_roi = mask[y0:y1, x0:x1]
hist = np.array([0, 0, 0., 0., 0., 0., 0., 255, 183.42105, 0., 0., 0., 0., 0., 0., 0.])
cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
hist = hist.reshape(-1)
# vis_roi = vis[y0:y1, x0:x1]
# cv.bitwise_not(vis_roi, vis_roi)
# vis[mask == 0] = 0












while True:
    ret, frame = cap.read()
    if ret == False:
        break
    # frame = scale(frame, factor=.25)

    vis = frame.copy()
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
    prob = cv.calcBackProject([hsv], [0], hist, [0, 180], 1)
    prob &= mask
    term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
    track_box, track_window = cv.CamShift(prob, track_window, term_crit)

    if show_backproj:
        vis[:] = prob[...,np.newaxis]
    try:
        cv.ellipse(vis, track_box, (0, 0, 255), 2)
    except:
        print(track_box)
    
    cv.imshow('camshift', vis)
    ch = cv.waitKey(5)
    if ch == 27:
        break
# cap.release()
cv.destroyAllWindows()

(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)
(800, 500, 850, 600)


In [26]:
cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
frame.shape

(1080, 1920, 3)

In [18]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
track_window
track_box, track_window = cv.CamShift(prob, track_window, term_crit)
cv.ellipse(vis, track_box, (0, 0, 255), 2)
cv.imshow('camshift', vis)
cv.waitKey(0)
cap.release()
cv.destroyAllWindows()

NameError: name 'prob' is not defined

In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture(1)

ret,frame = cap.read()
print(frame)
# setup initial location of window
r,h,c,w = 500, 50, 820, 100  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]

cv2.imshow('img2',roi)
cv2.waitKey(0)
cv2.destroyAllWindows()
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
test = roi_hist
print(roi_hist)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret, frame = cap.read()


    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        cv2.imshow('img2',img2)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

[[[5 1 1]
  [4 0 0]
  [4 0 0]
  ...
  [5 1 1]
  [4 2 2]
  [3 1 1]]

 [[5 1 1]
  [5 1 1]
  [5 1 1]
  ...
  [5 1 1]
  [3 1 1]
  [3 1 1]]

 [[5 1 1]
  [5 1 1]
  [4 0 0]
  ...
  [5 1 1]
  [3 1 1]
  [3 1 1]]

 ...

 [[3 1 1]
  [3 1 1]
  [7 3 3]
  ...
  [5 1 1]
  [2 0 0]
  [2 0 0]]

 [[5 1 1]
  [4 0 0]
  [6 2 2]
  ...
  [5 1 1]
  [2 0 0]
  [4 2 2]]

 [[5 1 1]
  [5 1 1]
  [5 1 1]
  ...
  [4 0 0]
  [2 0 0]
  [2 0 0]]]


error: C:\bld\opencv_1506447021968\work\opencv-3.3.0\modules\highgui\src\window.cpp:325: error: (-215) size.width>0 && size.height>0 in function cv::imshow


NameError: name 'np' is not defined

In [5]:
import numpy as np
import cv2

hard_code = np.array([[  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  3.5172415],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  1.7586207],
       [  1.7586207],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  7.034483 ],
       [  0.       ],
       [  8.793104 ],
       [  5.275862 ],
       [  5.275862 ],
       [ 21.103449 ],
       [ 21.103449 ],
       [ 15.827587 ],
       [ 14.068966 ],
       [ 58.034485 ],
       [ 80.89655  ],
       [170.58621  ],
       [128.37932  ],
       [149.48276  ],
       [128.37932  ],
       [153.       ],
       [154.75862  ],
       [133.65518  ],
       [255.       ],
       [168.82759  ],
       [133.65518  ],
       [ 84.413795 ],
       [ 91.44828  ],
       [ 77.37931  ],
       [ 28.137932 ],
       [  7.034483 ],
       [  3.5172415],
       [  0.       ],
       [ 24.62069  ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ]], dtype=np.float32)

cap = cv2.VideoCapture('../../vid/drone-p1.mp4')

# take first frame of the video
'''
n = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, n - 1)
'''
ret,frame = cap.read()
out = frame.copy()
# setup initial location of window
r,h,c,w = 500, 50, 820, 100  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]

roi_hist = hard_code

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
i = 0
while True:
    ret, frame = cap.read()
        
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        box_out = frame.copy()
        box_out2 = frame.copy()
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,255,0),2)
        cv2.imshow('img2',img2)
        x_pts, y_pts = pts.T[0], pts.T[1]
        x, y = int(np.mean(x_pts)), int(np.mean(y_pts))
        scale_down = 25
        box_size_half = 76
        cv2.rectangle(box_out,
                      (x - box_size_half, y - box_size_half + scale_down),
                      (x + box_size_half, y + box_size_half + scale_down),
                      (0,255,0),1)
        crop = box_out[y - 150 - box_size_half + scale_down + 1:y - 150 + box_size_half + scale_down - 1, x + 150 - box_size_half + 1:x + 150 + box_size_half - 1]
        crop2 = box_out[y + 300 - box_size_half + scale_down + 1:y + 300 + box_size_half + scale_down - 1, x + 150 - box_size_half + 1:x + 150 + box_size_half - 1]
        crop3 = box_out[150 - box_size_half + scale_down + 1:150 + box_size_half + scale_down - 1, x + 150 - box_size_half + 1:x + 150 + box_size_half - 1]
        cv2.imshow('crop', crop)
        cv2.imshow('crop2', crop2)
        cv2.imshow('crop3', crop3)
        #cv2.imwrite('not_drone{}.png'.format(i), crop)
        i += 1
        #cv2.imshow('box_out', box_out)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

In [58]:
np.mean(pts)

NameError: name 'avg' is not defined

In [27]:
frame.shape

(480, 640, 3)

In [47]:
frame = out.copy()
r,h,c,w = 200, 100, 300, 180  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]

cv2.imshow('img2',roi)
cv2.waitKey(0)
cv2.destroyAllWindows()
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
test = roi_hist

In [48]:
test

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.000000

In [53]:
import numpy as np
import cv2

hard_code = np.array([[  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  3.5172415],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  1.7586207],
       [  1.7586207],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  7.034483 ],
       [  0.       ],
       [  8.793104 ],
       [  5.275862 ],
       [  5.275862 ],
       [ 21.103449 ],
       [ 21.103449 ],
       [ 15.827587 ],
       [ 14.068966 ],
       [ 58.034485 ],
       [ 80.89655  ],
       [170.58621  ],
       [128.37932  ],
       [149.48276  ],
       [128.37932  ],
       [153.       ],
       [154.75862  ],
       [133.65518  ],
       [255.       ],
       [168.82759  ],
       [133.65518  ],
       [ 84.413795 ],
       [ 91.44828  ],
       [ 77.37931  ],
       [ 28.137932 ],
       [  7.034483 ],
       [  3.5172415],
       [  0.       ],
       [ 24.62069  ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ],
       [  0.       ]], dtype=np.float32)

cap = cv2.VideoCapture(1)

# take first frame of the video
n = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, n - 1)
ret,frame = cap.read()
out = frame.copy()
# setup initial location of window
r,h,c,w = 500, 50, 820, 100  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]

roi_hist = test

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret, frame = cap.read()
        
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        box_out = frame.copy()
        box_out2 = frame.copy()
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,255,0),2)
        cv2.imshow('img2',img2)
        x_pts, y_pts = pts.T[0], pts.T[1]
        x, y = int(np.mean(x_pts)), int(np.mean(y_pts))
        x_std, y_std = int(np.std(pts.T[0])), int(np.std(pts.T[1]))
        # print(x_std, y_std)
        
        x_box_size_half, y_box_size_half = 1 * x_std, 3 * y_std
        scale_down = 25
        box_size_half = 75
        cv2.rectangle(box_out,
                      (x - box_size_half, y - box_size_half + scale_down),
                      (x + box_size_half, y + box_size_half + scale_down),
                      (0,255,0),2)
        cv2.rectangle(box_out2,
                      (x - x_box_size_half, y - y_box_size_half + scale_down),
                      (x + x_box_size_half, y + y_box_size_half + scale_down),
                      (0,255,0),2)
        cv2.imshow('box_out', box_out)
        cv2.imshow('box_out2', box_out2)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()